# IS 733 Data Mining (Fall 2022)
#### Homework 3
#### Old Faithful Geyser Set
#### by Nadeem Ahmed

## Problem 1a
**Create and print out a scatter plot of this dataset, eruption time versus waiting time. (10 points)**

### Answer

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px # usig the Plotly plotting engine
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler # for standardizing the features
from ipywidgets import widgets # For interactive cluster and iteration selection

In [3]:
# Read data into Pandas dataframe
df = pd.read_csv('https://ruralatlasdataset.s3.amazonaws.com/faithful.csv')

#### Plot using Plotly library for interactions including hover information and 4 dimensional representation. Color scale shows the waiting time between eruptions and size of the points shows the eruption durations.

In [4]:
fig = px.scatter(df, x='eruptions', y='waiting', labels=dict(eruptions='Eruption Time',waiting='Waiting Time'), 
                size=df.eruptions, height=800, width=1200, color=df.waiting)
fig.update_layout(
    plot_bgcolor='black',
    title={
        'text': "Old Faithful Eruptions vs Waiting",
        'y':0.96,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

## Problem 1b
**How many clusters do you see based on your scatter plot? For the purposes of this question, a cluster is a “blob” of many data points that are close together, with regions of fewer data points between it and other “blobs”/clusters. (5 points)**

### Answer
We can see distinctly that there are 2 clusters patterns here. One where eruptions happen between 40-70 minutes and two where the eruptions happen between 70-100 minutes. The other pattern we can see from the color scale vs size of the marker points is that in the first cluster the eruptions are shorter (smaller circles) and the second cluster the eruptions are longer (larger circles). 

## Problem 1c
**Describe the steps of a hierarchical clustering algorithm. Based on your scatter plot, would this method be appropriate for this dataset? (10 points)**

### Answer
Hierarchical clustering is a option when we are not sure about the number of clusters we deduce from a scatter plot. If we use standard hierarchical clustering, it performs clustering in a bottom-up manner. It performs agglomerative clustering with basic steps as follows:

- First, make each instance in the dataset into a trivial mini-cluster
- Then, find the two closest clusters and merge them; repeat
- Clustering stops when all clusters have been merged into a single cluster 

The two standard algorithms for agglomerative bottoms-up hierarchical clustering are single linkage and complete linkage. Using single linkage, we compute the distances between the most similar members for each pair of clusters and merge the two clusters for which the distance between the most similar members is the smallest. The complete linkage approach is similar to single linkage but, instead of comparing the most similar members in each pair of clusters, we compare the most dissimilar members to perform the merge.

Looking at the scatter plot above, it is optically clear that there are at least two clusters. With k-means clustering we can use the fact that there is a possibility of two clusters and figure out the cluster boundaries or cluster membership of instances. If there was a more of wider spread in the instances in the scatter plot, we could try the hierarchical clustering algorithm to find the clusters or possibly hierarchy of clusters. 

## Problem 2
**Implement the k-means algorithm in Python, and use it to perform clustering on the Old Faithful dataset. Use the number of clusters that you identified in Problem 1. Be sure to ignore the first column, which contains instance ID numbers. In your notebook, including the following items:**

## Problem 2a
**Your source code for the k-means algorithm. You need to implement the algorithm from scratch. (45 points)**

### Answer
**FYI - There was no instance ID numbers in the dataset given as first column.**

In [5]:
# Start with standardizing the two features. Without standardizing, some of the outlier marker points are 
# confused as to which cluster they belong to and have dubious cluster membership.

std_scaler = MinMaxScaler()
data = std_scaler.fit_transform(df.to_numpy())
df_scaled = pd.DataFrame(data=data, columns=['eruptions','waiting']) # create a scaled version of the dataframe

In [6]:
df_scaled.head()

,eruptions,waiting
0,0.571429,0.679245
1,0.057143,0.207547
2,0.495143,0.584906
3,0.195143,0.358491
4,0.838000,0.792453


### Change cluster number and number of iterations as needed below in the next two cells and run the code cells below again for different results. 

In [7]:
# Interactive control to select number of clusters. 
# Defaults to TWO and max clusters for this dataset probably is FOUR and hence set to four. 
cluster_dropD = widgets.Dropdown(
    options=[('Two', 2), ('Three', 3),('Four', 4)],
    value=2,
    description='Clusters',
    disabled=False,
    )

# Display the control 
display(cluster_dropD)

Dropdown(description='Clusters', options=(('Two', 2), ('Three', 3), ('Four', 4)), value=2)

In [8]:
# Interactive control to select number of iterations. 
# Defaults to 8 which is pretty much a good point when expectation stabilizes.
# Minimum is set to 3 and max to 10 

num_of_iterations = widgets.BoundedIntText(
    value=8,
    min=3,
    max=10,
    step=1,
    description='Iterations:',
    disabled=False
)

# Display the control 
display(num_of_iterations)

BoundedIntText(value=8, description='Iterations:', max=10, min=3)

In [9]:
## initialize some needed variables
k = cluster_dropD.value 
expectation = np.zeros(shape=num_of_iterations.value)

#symbol map for plotly symbols for max four clusters
symmap = ['circle','square','diamond','cross'] 

# initialize random cluster centers (randomly sampled data points)
np.random.seed()
ridx = np.random.choice(range(len(data)), k, replace=False)
centroids = data[ridx,:]

In [10]:
# Plot the initial iteration 0 showing the initial centroids with no cluster assignment as of yet

fig = px.scatter(df_scaled, x='eruptions', y='waiting', labels=dict(eruptions='Eruption Time (scaled)',
                waiting='Waiting Time (scaled)', color="Wait Time"), 
                size=df.eruptions, height=800, width=1200, color=df.waiting)
                
fig.update_layout(
    plot_bgcolor='black',
    title={
        'text': "Iteration 0 with Random Centroids",
        'y':0.96,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.update_xaxes(showgrid=False, zeroline=False)
fig.update_yaxes(showgrid=False, zeroline=False)

# add random centroids
fig.add_trace(go.Scatter(x=centroids[:,0],y=centroids[:,1], name='Centroids', mode='markers', 
                  marker=dict(size=13, symbol="x-dot", color='red', line=dict(width=2, color="white"))))

fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01))
    
fig.show()

### IMPORTANT : Cluster symbols and colors might change during iteration plot print until semi-convergence. Have not figured out why Plotly behaves like that but rest assured the idea is conveyed. 

In [11]:
# loop over iterations
for iteri in range(num_of_iterations.value):
    
  # step 1: initialize and compute distances
  dists = np.zeros((data.shape[0],k))

  for ci in range(k):
    dists[:,ci] = np.sum((data-centroids[ci,:])**2,axis=1) # calculate squared euclidian distances the numpy way
  
  # step 2: assign to cluster based on minimum distance
  clusterid = np.argmin(dists,axis=1)
  

  # sum of all minimum distances to the centroids for all clusters for this iteration
  cluster_min = np.min(dists,axis=1)
  expectation[iteri] = np.sum(cluster_min) 
    
  # step 3: recompute centers
  for ki in range(k):
    centroids[ki,:] = [ np.mean(data[clusterid==ki,0]), np.mean(data[clusterid==ki,1]) ]

  # assign a new column with cluster ID to scaled dataframe
  df_scaled['clusterid'] = clusterid

  ## IMPORTANT : Cluster symbols and colors might change during iteration plot print but cluster membership idea conveyed
  
  # plot the scaled dataframe with cluster membership 
  fig = px.scatter(df_scaled, x='eruptions', y='waiting', labels=dict(eruptions='Eruption Time (scaled)',waiting='Waiting Time (scaled)',
                    clusterid='Cluster ID'), size=df.eruptions, height=800, width=1200, color=df_scaled.clusterid, 
                    symbol=df_scaled.clusterid, symbol_sequence=symmap)

  # plot the new centroids  
  fig.add_trace(go.Scatter(x=centroids[:,0],y=centroids[:,1], name='Centroids', mode='markers', 
                  marker=dict(size=13, symbol="x-dot", color='red', line=dict(width=2, color="white"))))
  
  fig.update_layout(
    plot_bgcolor='darkslategrey',
    title={
        'text': f"Iteration {iteri + 1}",
        'y':0.96,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    coloraxis_showscale=False
        )
  
  fig.update_xaxes(showgrid=False, zeroline=False)
  fig.update_yaxes(showgrid=False, zeroline=False)
  
  fig.show()


## Problem 2b
**A scatter plot of your final clustering, with the data points in each cluster color-coded, or plotted with different symbols. Include the cluster centers in your plot. (10 points)**

### Answer

Last plot from above answer 2a has the stable cluster plot.


In [12]:
# last plot from above answer has the stable cluster plot

fig.show()

## Problem 2c
**A plot of the k-means objective function versus iterations of the algorithm.**

### Answer

In [13]:
fig = px.line(x=np.arange(num_of_iterations.value)+1, y=expectation, markers=True, labels=dict(x='Iteration', y='Expectation'), height=600,
    width=900)
fig.update_layout(
    plot_bgcolor='black',
    title={
        'text': "Expectation Vs Iteration",
        'y':0.96,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'}
  )
fig.update_traces(line_color='red', line_width=2)
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

## Problem 2d
**Did the method manage to find the clusters that you identified in Problem 1? If not, did it help to run the method again with another random initialization? (10 points)**

### Answer

Yes, the method employed above was able to find the two clusters. But experimenting with different cluster values of 3 and 4 and different iteration runs (by changing the interactive controls), there could be very subtle argument made for a possible sparsely populated 3rd cluster which is between 60- and 80-minute eruption time. But it all depends on where the initial 3rd centroid falls randomly. But generally speaking, there are two distinct clusters as discussed earlier and this method was able to identify those two clusters. 